# Compare TF Models Trained on Different Feature Sets

Trains 15 different models with the same architecture on variations of the same data. These variations include:
- Variations on how NaN values were treated in the dataset:
  - dropped entirely,
  - imputed using mean for some and median for highly skewed features (and mode for non-numeric features), or
  - imputed using mean for all features (and mode for non-numeric features).
- Variations of which features were included to train the model:
  - minimal data (location, time)
  - baseline data (location, time, immediate weather data),
  - baseline data plus long-term weather data,
  - baseline data plus fire history data, or
  - baseline data plus long-term weather and fire history data.

Each model is trained with a different pairing of data imputation and feature set.

The train-test split is the same for each variation of features within one data imputation approach.

Compares the performances of the models.

## Setup
Imports, read in data

In [125]:
import numpy as np  # linear algebra
import pandas as pd  # data processing
import matplotlib.pyplot as plt  # plotting

import math
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings

import tensorflow as tf  # model

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
DATASET_FILE_DROPPED_NANS = "/content/drive/My Drive/ML6140  - Project/Data/enhanced_data_df1.csv"
DATASET_FILE_IMPUTED_MEAN_ONLY = "/content/drive/My Drive/ML6140  - Project/Data/enhanced_data_df3.csv"
DATASET_FILE_IMPUTED_MEAN_MEDIAN = "/content/drive/My Drive/ML6140  - Project/Data/enhanced_data_df2.csv"

In [ ]:
drop_cols = ["incident_name", "GHCN_station_id", "LCD_station_id",
                      "mode_functionSkyConditions", "10_mode_functionSkyConditions",
                      "30_mode_functionSkyConditions",
                      "60_mode_functionSkyConditions", "incident_geohash",
                      "LCD_station_elevation", "LCD_station_distance",
                      "GHCN_station_elevation", "GHCN_station_distance",
             'dist_mode_functionSkyConditions',]

In [ ]:
data_dropped_nans = pd.read_csv(DATASET_FILE_DROPPED_NANS).drop(drop_cols, axis=1)
data_imputed_mean_only = pd.read_csv(DATASET_FILE_IMPUTED_MEAN_ONLY).drop(drop_cols, axis=1)
data_imputed_mean_median = pd.read_csv(DATASET_FILE_IMPUTED_MEAN_MEDIAN).drop(drop_cols, axis=1)

In [ ]:
data_dropped_nans.info()

In [ ]:
minimal_features = [
    "incident_created_year",
    "incident_created_month",
    "incident_created_day",
    "incident_created_hour",
    "incident_created_minute",
    "incident_latitude",
    "incident_longitude"
]

In [ ]:
baseline_features = [
  "incident_created_year",
  "incident_created_month",
  "incident_created_day",
  "incident_created_hour",
  "incident_created_minute",
  "incident_latitude",
  "incident_longitude",
  "sumPrecipitation",
  "dist_sumPrecipitation",
  "maxDryBulbTemperature",
  "dist_maxDryBulbTemperature",
  "minDryBulbTemperature",
  "dist_minDryBulbTemperature",
  "meanDryBulbTemperature",
  "dist_meanDryBulbTemperature",
  "meanDewPointTemperature",
  "dist_meanDewPointTemperature",
  "meanWetBulbTemperature",
  "dist_meanWetBulbTemperature",
  "meanWindSpeed",
  "dist_meanWindSpeed",
  "meanRelativeHumidity",
  "dist_meanRelativeHumidity",
  "minRelativeHumidity",
  "dist_minRelativeHumidity",
  "maxRelativeHumidity",
  "dist_maxRelativeHumidity",
  "maxWindSpeed",
  "dist_maxWindSpeed",
  "calculate_circular_meanWindDirection",
  "dist_calculate_circular_meanWindDirection",
  "mode_functionWindDirection",
  "dist_mode_functionWindDirection",
]

In [ ]:
weather_history_features = [
  "10_sumPrecipitation",
  "10_maxDryBulbTemperature",
  "10_minDryBulbTemperature",
  "10_meanDryBulbTemperature",
  "10_meanDewPointTemperature",
  "10_meanWetBulbTemperature",
  "10_meanWindSpeed",
  "10_meanRelativeHumidity",
  "10_minRelativeHumidity",
  "10_maxRelativeHumidity",
  "10_maxWindSpeed",
  "10_calculate_circular_meanWindDirection",
  "10_mode_functionWindDirection",
  "30_sumPrecipitation",
  "30_maxDryBulbTemperature",
  "30_minDryBulbTemperature",
  "30_meanDryBulbTemperature",
  "30_meanDewPointTemperature",
  "30_meanWetBulbTemperature",
  "30_meanWindSpeed",
  "30_meanRelativeHumidity",
  "30_minRelativeHumidity",
  "30_maxRelativeHumidity",
  "30_maxWindSpeed",
  "30_calculate_circular_meanWindDirection",
  "30_mode_functionWindDirection",
  "60_sumPrecipitation",
  "60_maxDryBulbTemperature",
  "60_minDryBulbTemperature",
  "60_meanDryBulbTemperature",
  "60_meanDewPointTemperature",
  "60_meanWetBulbTemperature",
  "60_meanWindSpeed",
  "60_meanRelativeHumidity",
  "60_minRelativeHumidity",
  "60_maxRelativeHumidity",
  "60_maxWindSpeed",
  "60_calculate_circular_meanWindDirection",
  "60_mode_functionWindDirection"
]

In [ ]:
fire_history_features = [
  "far_hist_avg_acres_burned",
  "near_hist_avg_acres_burned"
]

In [ ]:
def get_experiment_data(dataframe):
  # So the split is the same across each experimental feature set.
  train, validate = train_test_split(dataframe)
  train_x = train.drop(["class_label"], axis=1)
  train_y = train["class_label"]
  validate_x = validate.drop(["class_label"], axis=1)
  validate_y = validate["class_label"]

  data_dict = {}
  data_dict["train_y"] = train_y.values
  data_dict["validate_y"] = validate_y.values
  data_dict["train_x"] = {}
  data_dict["validate_x"] = {}

  data_dict["train_x"]["minimal"] = train_x[minimal_features].values
  data_dict["validate_x"]["minimal"] = validate_x[minimal_features].values

  data_dict["train_x"]["baseline"] = train_x[baseline_features].values
  data_dict["validate_x"]["baseline"] = validate_x[baseline_features].values

  data_dict["train_x"]["weather_hist"] = train_x[baseline_features
                                                 + weather_history_features].values
  data_dict["validate_x"]["weather_hist"] = validate_x[baseline_features
                                                       + weather_history_features].values

  data_dict["train_x"]["fire_hist"] = train_x[baseline_features
                                              + fire_history_features].values
  data_dict["validate_x"]["fire_hist"] = validate_x[baseline_features
                                                    + fire_history_features].values

  data_dict["train_x"]["all"] = train_x.values
  data_dict["validate_x"]["all"] = validate_x.values

  return data_dict

In [ ]:
class Metric:
  def __init__(self, y_array, pred_y_array):
    self.y_array = y_array
    self.pred_y_array = pred_y_array

    df = pd.concat([pd.DataFrame(y_array, columns=["y"]),
                  pd.DataFrame(pred_y_array, columns=["pred_y"])],
                 axis=1)

    self.tp = len(df[(df.y == 1) & (df.pred_y == 1)])
    self.tn = len(df[(df.y == 0) & (df.pred_y == 0)])
    self.fp = len(df[(df.y == 0) & (df.pred_y == 1)])
    self.fn = len(df[(df.y == 1) & (df.pred_y == 0)])
    self.total = len(df)

  def true_positive_rate(self):
    try:
      return self.tp / self.total
    except ZeroDivisionError:
      return np.nan


  def true_negative_rate(self):
    try:
      return self.tn / self.total
    except ZeroDivisionError:
      return np.nan


  def false_positive_rate(self):
    try:
      return self.fp / self.total
    except ZeroDivisionError:
      return np.nan


  def false_negative_rate(self):
    try:
      return self.fn / self.total
    except ZeroDivisionError:
      return np.nan


  def confusion_matrix(self, rates=False):
    if rates:
      return self._confusion_matrix(self.true_positive_rate(),
                                    self.true_negative_rate(),
                                    self.false_positive_rate(),
                                    self.false_negative_rate())
    return self._confusion_matrix(self.tp, self.tn, self.fp, self.fn)


  def _confusion_matrix(self, tp, tn, fp, fn):
    mat = pd.DataFrame([[tp, fn], [fp, tn]],
                       columns=["Predicted Positive", "Predicted Negative"],
                       index=["Positive", "Negative"])
    return mat


  def precision(self):
    try:
      den = self.tp + self.fp
      return self.tp / den
    except ZeroDivisionError:
      return np.nan


  def recall(self):
    try:
      den = self.tp + self.fn
      return self.tp / den
    except ZeroDivisionError:
      return np.nan


  def accuracy(self):
    try:
      trues = self.tp + self.tn
      return trues / self.total
    except ZeroDivisionError:
      return np.nan


  def f1_score(self):
    try:
      den = 2 * self.tp + self.fp + self.fn
      return 2 * self.tp / den
    except ZeroDivisionError:
      return np.nan

## Train Models

### DF 1: All NaNs dropped

In [ ]:
df1_data = get_experiment_data(data_dropped_nans)

#### Minimal

In [ ]:
df1_minimal_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df1_minimal_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df1_minimal_history = df1_minimal_model.fit(df1_data["train_x"]["minimal"], df1_data["train_y"], epochs=300)

Epoch 1/300
115/115 [==============================] - 3s 11ms/step - loss: 2.5499
Epoch 2/300
115/115 [==============================] - 1s 11ms/step - loss: 0.7316
Epoch 3/300
115/115 [==============================] - 1s 10ms/step - loss: 0.7100
Epoch 4/300
115/115 [==============================] - 1s 11ms/step - loss: 0.7993
Epoch 5/300
115/115 [==============================] - 1s 9ms/step - loss: 0.6858
Epoch 6/300
115/115 [==============================] - 1s 7ms/step - loss: 0.6531
Epoch 7/300
115/115 [==============================] - 1s 7ms/step - loss: 0.6692
Epoch 8/300
115/115 [==============================] - 1s 7ms/step - loss: 0.6439
Epoch 9/300
115/115 [==============================] - 1s 8ms/step - loss: 0.6445
Epoch 10/300
115/115 [==============================] - 1s 7ms/step - loss: 0.6453
Epoch 11/300
115/115 [==============================] - 1s 7ms/step - loss: 0.6448
Epoch 12/300
115/115 [==============================] - 1s 7ms/step - loss: 0.6456
Epoch 13/

In [ ]:
df1_minimal_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               4096      
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2

In [ ]:
df1_minimal_pred_y = df1_minimal_model.predict(df1_data["validate_x"]["minimal"])
df1_minimal_pred_y = np.where(df1_minimal_pred_y > .5, 1, 0)

39/39 [==============================] - 0s 4ms/step


In [ ]:
df1_minimal_model_metric = Metric(df1_data["validate_y"], df1_minimal_pred_y)
print(f"Precision: {df1_minimal_model_metric.precision()}")
print(f"Recall: {df1_minimal_model_metric.recall()}")
print(f"Accuracy: {df1_minimal_model_metric.accuracy()}")
print(f"F1 Score: {df1_minimal_model_metric.f1_score()}")
display(df1_minimal_model_metric.confusion_matrix())

Precision: nan
Recall: 0.0
Accuracy: 0.6972176759410802
F1 Score: 0.0


,Predicted Positive,Predicted Negative
Positive,0,370
Negative,0,852


#### Baseline

In [ ]:
df1_baseline_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df1_baseline_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df1_baseline_history = df1_baseline_model.fit(df1_data["train_x"]["baseline"], df1_data["train_y"], epochs=300)

Epoch 1/300
115/115 [==============================] - 3s 11ms/step - loss: 1.7815
Epoch 2/300
115/115 [==============================] - 1s 11ms/step - loss: 0.6283
Epoch 3/300
115/115 [==============================] - 1s 8ms/step - loss: 0.6387
Epoch 4/300
115/115 [==============================] - 1s 7ms/step - loss: 0.5902
Epoch 5/300
115/115 [==============================] - 1s 7ms/step - loss: 0.5725
Epoch 6/300
115/115 [==============================] - 1s 8ms/step - loss: 0.5590
Epoch 7/300
115/115 [==============================] - 1s 8ms/step - loss: 0.5445
Epoch 8/300
115/115 [==============================] - 1s 8ms/step - loss: 0.5351
Epoch 9/300
115/115 [==============================] - 1s 8ms/step - loss: 0.5495
Epoch 10/300
115/115 [==============================] - 1s 8ms/step - loss: 0.5205
Epoch 11/300
115/115 [==============================] - 1s 8ms/step - loss: 0.5213
Epoch 12/300
115/115 [==============================] - 1s 8ms/step - loss: 0.5111
Epoch 13/30

In [ ]:
df1_baseline_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               17408     
                                                                 
 dense_9 (Dense)             (None, 512)               262656    
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 dense_11 (Dense)            (None, 256)               65792     
                                                                 
 dense_12 (Dense)            (None, 128)               32896     
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 32)               

In [ ]:
df1_baseline_pred_y = df1_baseline_model.predict(df1_data["validate_x"]["baseline"])
df1_baseline_pred_y = np.where(df1_baseline_pred_y > .5, 1, 0)

39/39 [==============================] - 0s 2ms/step


In [ ]:
df1_baseline_model_metric = Metric(df1_data["validate_y"], df1_baseline_pred_y)
print(f"Precision: {df1_baseline_model_metric.precision()}")
print(f"Recall: {df1_baseline_model_metric.recall()}")
print(f"Accuracy: {df1_baseline_model_metric.accuracy()}")
print(f"F1 Score: {df1_baseline_model_metric.f1_score()}")
display(df1_baseline_model_metric.confusion_matrix())

Precision: 0.5858310626702997
Recall: 0.581081081081081
Accuracy: 0.7487725040916531
F1 Score: 0.5834464043419267


,Predicted Positive,Predicted Negative
Positive,215,155
Negative,152,700


#### With Weather History

In [ ]:
df1_weather_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df1_weather_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df1_weather_history = df1_weather_model.fit(df1_data["train_x"]["weather_hist"], df1_data["train_y"], epochs=300)

Epoch 1/300
115/115 [==============================] - 2s 8ms/step - loss: 1.6077
Epoch 2/300
115/115 [==============================] - 1s 8ms/step - loss: 0.6204
Epoch 3/300
115/115 [==============================] - 1s 8ms/step - loss: 0.5530
Epoch 4/300
115/115 [==============================] - 1s 7ms/step - loss: 0.5411
Epoch 5/300
115/115 [==============================] - 1s 7ms/step - loss: 0.5161
Epoch 6/300
115/115 [==============================] - 1s 9ms/step - loss: 0.5060
Epoch 7/300
115/115 [==============================] - 1s 11ms/step - loss: 0.4992
Epoch 8/300
115/115 [==============================] - 1s 10ms/step - loss: 0.4805
Epoch 9/300
115/115 [==============================] - 1s 11ms/step - loss: 0.4804
Epoch 10/300
115/115 [==============================] - 1s 11ms/step - loss: 0.4823
Epoch 11/300
115/115 [==============================] - 1s 10ms/step - loss: 0.4772
Epoch 12/300
115/115 [==============================] - 1s 8ms/step - loss: 0.4590
Epoch 13

In [ ]:
df1_weather_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 512)               37376     
                                                                 
 dense_17 (Dense)            (None, 512)               262656    
                                                                 
 dense_18 (Dense)            (None, 256)               131328    
                                                                 
 dense_19 (Dense)            (None, 256)               65792     
                                                                 
 dense_20 (Dense)            (None, 128)               32896     
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dense_22 (Dense)            (None, 32)               

In [ ]:
df1_weather_pred_y = df1_weather_model.predict(df1_data["validate_x"]["weather_hist"])
df1_weather_pred_y = np.where(df1_weather_pred_y > .5, 1, 0)

39/39 [==============================] - 0s 4ms/step


In [ ]:
df1_weather_model_metric = Metric(df1_data["validate_y"], df1_weather_pred_y)
print(f"Precision: {df1_weather_model_metric.precision()}")
print(f"Recall: {df1_weather_model_metric.recall()}")
print(f"Accuracy: {df1_weather_model_metric.accuracy()}")
print(f"F1 Score: {df1_weather_model_metric.f1_score()}")
display(df1_weather_model_metric.confusion_matrix())

Precision: 0.5685279187817259
Recall: 0.6054054054054054
Accuracy: 0.7414075286415712
F1 Score: 0.5863874345549738


,Predicted Positive,Predicted Negative
Positive,224,146
Negative,170,682


#### With Fire History

In [ ]:
df1_fire_hist_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df1_fire_hist_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df1_fire_hist_history = df1_fire_hist_model.fit(df1_data["train_x"]["fire_hist"], df1_data["train_y"], epochs=300)

Epoch 1/300
115/115 [==============================] - 2s 7ms/step - loss: 4.3956
Epoch 2/300
115/115 [==============================] - 1s 7ms/step - loss: 1.6341
Epoch 3/300
115/115 [==============================] - 1s 7ms/step - loss: 0.9939
Epoch 4/300
115/115 [==============================] - 1s 7ms/step - loss: 0.7949
Epoch 5/300
115/115 [==============================] - 1s 7ms/step - loss: 0.5947
Epoch 6/300
115/115 [==============================] - 1s 7ms/step - loss: 0.5202
Epoch 7/300
115/115 [==============================] - 1s 7ms/step - loss: 0.4463
Epoch 8/300
115/115 [==============================] - 1s 7ms/step - loss: 0.4928
Epoch 9/300
115/115 [==============================] - 1s 7ms/step - loss: 0.4565
Epoch 10/300
115/115 [==============================] - 1s 7ms/step - loss: 0.4456
Epoch 11/300
115/115 [==============================] - 1s 8ms/step - loss: 0.4164
Epoch 12/300
115/115 [==============================] - 1s 10ms/step - loss: 0.4465
Epoch 13/300

In [ ]:
df1_fire_hist_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               18432     
                                                                 
 dense_25 (Dense)            (None, 512)               262656    
                                                                 
 dense_26 (Dense)            (None, 256)               131328    
                                                                 
 dense_27 (Dense)            (None, 256)               65792     
                                                                 
 dense_28 (Dense)            (None, 128)               32896     
                                                                 
 dense_29 (Dense)            (None, 64)                8256      
                                                                 
 dense_30 (Dense)            (None, 32)               

In [ ]:
df1_fire_hist_pred_y = df1_fire_hist_model.predict(df1_data["validate_x"]["fire_hist"])
df1_fire_hist_pred_y = np.where(df1_fire_hist_pred_y > .5, 1, 0)

39/39 [==============================] - 0s 2ms/step


In [ ]:
df1_fire_hist_model_metric = Metric(df1_data["validate_y"], df1_fire_hist_pred_y)
print(f"Precision: {df1_fire_hist_model_metric.precision()}")
print(f"Recall: {df1_fire_hist_model_metric.recall()}")
print(f"Accuracy: {df1_fire_hist_model_metric.accuracy()}")
print(f"F1 Score: {df1_fire_hist_model_metric.f1_score()}")
display(df1_fire_hist_model_metric.confusion_matrix())

Precision: 0.7447368421052631
Recall: 0.7648648648648648
Accuracy: 0.8494271685761048
F1 Score: 0.7546666666666667


,Predicted Positive,Predicted Negative
Positive,283,87
Negative,97,755


#### With All Features

In [ ]:
df1_all_features_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df1_all_features_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df1_all_features_history = df1_all_features_model.fit(df1_data["train_x"]["all"], df1_data["train_y"], epochs=300)

Epoch 1/300
115/115 [==============================] - 2s 8ms/step - loss: 4.9996
Epoch 2/300
115/115 [==============================] - 1s 7ms/step - loss: 1.5773
Epoch 3/300
115/115 [==============================] - 1s 7ms/step - loss: 0.7487
Epoch 4/300
115/115 [==============================] - 1s 8ms/step - loss: 0.8677
Epoch 5/300
115/115 [==============================] - 1s 11ms/step - loss: 0.8101
Epoch 6/300
115/115 [==============================] - 1s 11ms/step - loss: 0.5405
Epoch 7/300
115/115 [==============================] - 1s 11ms/step - loss: 0.5821
Epoch 8/300
115/115 [==============================] - 1s 12ms/step - loss: 0.4528
Epoch 9/300
115/115 [==============================] - 1s 11ms/step - loss: 0.4413
Epoch 10/300
115/115 [==============================] - 1s 8ms/step - loss: 0.3998
Epoch 11/300
115/115 [==============================] - 1s 7ms/step - loss: 0.4348
Epoch 12/300
115/115 [==============================] - 1s 8ms/step - loss: 0.4541
Epoch 13

In [ ]:
df1_all_features_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 512)               38400     
                                                                 
 dense_33 (Dense)            (None, 512)               262656    
                                                                 
 dense_34 (Dense)            (None, 256)               131328    
                                                                 
 dense_35 (Dense)            (None, 256)               65792     
                                                                 
 dense_36 (Dense)            (None, 128)               32896     
                                                                 
 dense_37 (Dense)            (None, 64)                8256      
                                                                 
 dense_38 (Dense)            (None, 32)               

In [ ]:
df1_all_features_pred_y = df1_all_features_model.predict(df1_data["validate_x"]["all"])
df1_all_features_pred_y = np.where(df1_all_features_pred_y > .5, 1, 0)

39/39 [==============================] - 0s 3ms/step


In [ ]:
df1_all_features_model_metric = Metric(df1_data["validate_y"], df1_all_features_pred_y)
print(f"Precision: {df1_all_features_model_metric.precision()}")
print(f"Recall: {df1_all_features_model_metric.recall()}")
print(f"Accuracy: {df1_all_features_model_metric.accuracy()}")
print(f"F1 Score: {df1_all_features_model_metric.f1_score()}")
display(df1_all_features_model_metric.confusion_matrix())

Precision: 0.7885117493472585
Recall: 0.8162162162162162
Accuracy: 0.8780687397708674
F1 Score: 0.8021248339973439


,Predicted Positive,Predicted Negative
Positive,302,68
Negative,81,771


### DF 2: Impute with mean and median

In [ ]:
df2_data = get_experiment_data(data_imputed_mean_median)

#### Minimal

In [ ]:
df2_minimal_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df2_minimal_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df2_minimal_history = df2_minimal_model.fit(df2_data["train_x"]["minimal"], df2_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 2s 7ms/step - loss: 2.3996
Epoch 2/300
137/137 [==============================] - 1s 7ms/step - loss: 0.7374
Epoch 3/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6782
Epoch 4/300
137/137 [==============================] - 1s 8ms/step - loss: 0.6509
Epoch 5/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6459
Epoch 6/300
137/137 [==============================] - 1s 9ms/step - loss: 0.6673
Epoch 7/300
137/137 [==============================] - 2s 11ms/step - loss: 0.6530
Epoch 8/300
137/137 [==============================] - 1s 11ms/step - loss: 0.6485
Epoch 9/300
137/137 [==============================] - 2s 11ms/step - loss: 0.6399
Epoch 10/300
137/137 [==============================] - 1s 11ms/step - loss: 0.6451
Epoch 11/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6528
Epoch 12/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6379
Epoch 13/

In [ ]:
df2_minimal_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 512)               4096      
                                                                 
 dense_41 (Dense)            (None, 512)               262656    
                                                                 
 dense_42 (Dense)            (None, 256)               131328    
                                                                 
 dense_43 (Dense)            (None, 256)               65792     
                                                                 
 dense_44 (Dense)            (None, 128)               32896     
                                                                 
 dense_45 (Dense)            (None, 64)                8256      
                                                                 
 dense_46 (Dense)            (None, 32)               

In [ ]:
df2_minimal_pred_y = df2_minimal_model.predict(df2_data["validate_x"]["minimal"])
df2_minimal_pred_y = np.where(df2_minimal_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 3ms/step


In [ ]:
df2_minimal_model_metric = Metric(df2_data["validate_y"], df2_minimal_pred_y)
print(f"Precision: {df2_minimal_model_metric.precision()}")
print(f"Recall: {df2_minimal_model_metric.recall()}")
print(f"Accuracy: {df2_minimal_model_metric.accuracy()}")
print(f"F1 Score: {df2_minimal_model_metric.f1_score()}")
display(df2_minimal_model_metric.confusion_matrix())

Precision: nan
Recall: 0.0
Accuracy: 0.659106529209622
F1 Score: 0.0


,Predicted Positive,Predicted Negative
Positive,0,496
Negative,0,959


#### Baseline

In [ ]:
df2_baseline_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df2_baseline_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df2_baseline_history = df2_baseline_model.fit(df2_data["train_x"]["baseline"], df2_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 3s 11ms/step - loss: 1.4310
Epoch 2/300
137/137 [==============================] - 1s 11ms/step - loss: 0.6967
Epoch 3/300
137/137 [==============================] - 1s 10ms/step - loss: 0.5795
Epoch 4/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5741
Epoch 5/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5444
Epoch 6/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5302
Epoch 7/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5255
Epoch 8/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5147
Epoch 9/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5161
Epoch 10/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5230
Epoch 11/300
137/137 [==============================] - 1s 8ms/step - loss: 0.5086
Epoch 12/300
137/137 [==============================] - 1s 8ms/step - loss: 0.5082
Epoch 13/3

In [ ]:
df2_baseline_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 512)               17408     
                                                                 
 dense_49 (Dense)            (None, 512)               262656    
                                                                 
 dense_50 (Dense)            (None, 256)               131328    
                                                                 
 dense_51 (Dense)            (None, 256)               65792     
                                                                 
 dense_52 (Dense)            (None, 128)               32896     
                                                                 
 dense_53 (Dense)            (None, 64)                8256      
                                                                 
 dense_54 (Dense)            (None, 32)               

In [ ]:
df2_baseline_pred_y = df2_baseline_model.predict(df2_data["validate_x"]["baseline"])
df2_baseline_pred_y = np.where(df2_baseline_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 2ms/step


In [ ]:
df2_baseline_model_metric = Metric(df2_data["validate_y"], df2_baseline_pred_y)
print(f"Precision: {df2_baseline_model_metric.precision()}")
print(f"Recall: {df2_baseline_model_metric.recall()}")
print(f"Accuracy: {df2_baseline_model_metric.accuracy()}")
print(f"F1 Score: {df2_baseline_model_metric.f1_score()}")
display(df2_baseline_model_metric.confusion_matrix())

Precision: 0.6557734204793029
Recall: 0.6068548387096774
Accuracy: 0.7573883161512027
F1 Score: 0.6303664921465969


,Predicted Positive,Predicted Negative
Positive,301,195
Negative,158,801


#### With Weather History

In [ ]:
df2_weather_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df2_weather_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df2_weather_history = df2_weather_model.fit(df2_data["train_x"]["weather_hist"], df2_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 3s 8ms/step - loss: 1.5907
Epoch 2/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5789
Epoch 3/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5373
Epoch 4/300
137/137 [==============================] - 1s 10ms/step - loss: 0.5230
Epoch 5/300
137/137 [==============================] - 1s 10ms/step - loss: 0.4984
Epoch 6/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4902
Epoch 7/300
137/137 [==============================] - 1s 10ms/step - loss: 0.4890
Epoch 8/300
137/137 [==============================] - 1s 10ms/step - loss: 0.4784
Epoch 9/300
137/137 [==============================] - 1s 8ms/step - loss: 0.4653
Epoch 10/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4807
Epoch 11/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4655
Epoch 12/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4599
Epoch 13

In [ ]:
df2_weather_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            (None, 512)               37376     
                                                                 
 dense_57 (Dense)            (None, 512)               262656    
                                                                 
 dense_58 (Dense)            (None, 256)               131328    
                                                                 
 dense_59 (Dense)            (None, 256)               65792     
                                                                 
 dense_60 (Dense)            (None, 128)               32896     
                                                                 
 dense_61 (Dense)            (None, 64)                8256      
                                                                 
 dense_62 (Dense)            (None, 32)               

In [ ]:
df2_weather_pred_y = df2_weather_model.predict(df2_data["validate_x"]["weather_hist"])
df2_weather_pred_y = np.where(df2_weather_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 2ms/step


In [ ]:
df2_weather_model_metric = Metric(df2_data["validate_y"], df2_weather_pred_y)
print(f"Precision: {df2_weather_model_metric.precision()}")
print(f"Recall: {df2_weather_model_metric.recall()}")
print(f"Accuracy: {df2_weather_model_metric.accuracy()}")
print(f"F1 Score: {df2_weather_model_metric.f1_score()}")
display(df2_weather_model_metric.confusion_matrix())

Precision: 0.6756756756756757
Recall: 0.6048387096774194
Accuracy: 0.7663230240549829
F1 Score: 0.6382978723404256


,Predicted Positive,Predicted Negative
Positive,300,196
Negative,144,815


#### With Fire History

In [ ]:
df2_fire_hist_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df2_fire_hist_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df2_fire_hist_history = df2_fire_hist_model.fit(df2_data["train_x"]["fire_hist"], df2_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 2s 7ms/step - loss: 3.3044
Epoch 2/300
137/137 [==============================] - 1s 7ms/step - loss: 1.0718
Epoch 3/300
137/137 [==============================] - 1s 7ms/step - loss: 0.7620
Epoch 4/300
137/137 [==============================] - 1s 8ms/step - loss: 0.5465
Epoch 5/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4973
Epoch 6/300
137/137 [==============================] - 1s 8ms/step - loss: 0.4869
Epoch 7/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4990
Epoch 8/300
137/137 [==============================] - 1s 10ms/step - loss: 0.4591
Epoch 9/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4325
Epoch 10/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4373
Epoch 11/300
137/137 [==============================] - 1s 9ms/step - loss: 0.4218
Epoch 12/300
137/137 [==============================] - 1s 8ms/step - loss: 0.4309
Epoch 13/

In [ ]:
df2_fire_hist_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_64 (Dense)            (None, 512)               18432     
                                                                 
 dense_65 (Dense)            (None, 512)               262656    
                                                                 
 dense_66 (Dense)            (None, 256)               131328    
                                                                 
 dense_67 (Dense)            (None, 256)               65792     
                                                                 
 dense_68 (Dense)            (None, 128)               32896     
                                                                 
 dense_69 (Dense)            (None, 64)                8256      
                                                                 
 dense_70 (Dense)            (None, 32)               

In [ ]:
df2_fire_hist_pred_y = df2_fire_hist_model.predict(df2_data["validate_x"]["fire_hist"])
df2_fire_hist_pred_y = np.where(df2_fire_hist_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 2ms/step


In [ ]:
df2_fire_hist_model_metric = Metric(df2_data["validate_y"], df2_fire_hist_pred_y)
print(f"Precision: {df2_fire_hist_model_metric.precision()}")
print(f"Recall: {df2_fire_hist_model_metric.recall()}")
print(f"Accuracy: {df2_fire_hist_model_metric.accuracy()}")
print(f"F1 Score: {df2_fire_hist_model_metric.f1_score()}")
display(df2_fire_hist_model_metric.confusion_matrix())

Precision: 0.8019607843137255
Recall: 0.8245967741935484
Accuracy: 0.8707903780068729
F1 Score: 0.8131212723658051


,Predicted Positive,Predicted Negative
Positive,409,87
Negative,101,858


#### With All Features

In [ ]:
df2_all_features_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df2_all_features_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df2_all_features_history = df2_all_features_model.fit(df2_data["train_x"]["all"], df2_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 3s 10ms/step - loss: 3.8194
Epoch 2/300
137/137 [==============================] - 1s 10ms/step - loss: 0.9396
Epoch 3/300
137/137 [==============================] - 1s 11ms/step - loss: 1.8719
Epoch 4/300
137/137 [==============================] - 1s 10ms/step - loss: 0.5947
Epoch 5/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5817
Epoch 6/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4946
Epoch 7/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4446
Epoch 8/300
137/137 [==============================] - 1s 8ms/step - loss: 0.3883
Epoch 9/300
137/137 [==============================] - 1s 8ms/step - loss: 0.3787
Epoch 10/300
137/137 [==============================] - 1s 7ms/step - loss: 0.3864
Epoch 11/300
137/137 [==============================] - 1s 7ms/step - loss: 0.3732
Epoch 12/300
137/137 [==============================] - 1s 7ms/step - loss: 0.3543
Epoch 13/

In [ ]:
df2_all_features_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_72 (Dense)            (None, 512)               38400     
                                                                 
 dense_73 (Dense)            (None, 512)               262656    
                                                                 
 dense_74 (Dense)            (None, 256)               131328    
                                                                 
 dense_75 (Dense)            (None, 256)               65792     
                                                                 
 dense_76 (Dense)            (None, 128)               32896     
                                                                 
 dense_77 (Dense)            (None, 64)                8256      
                                                                 
 dense_78 (Dense)            (None, 32)               

In [ ]:
df2_all_features_pred_y = df2_all_features_model.predict(df2_data["validate_x"]["all"])
df2_all_features_pred_y = np.where(df2_all_features_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 2ms/step


In [ ]:
df2_all_features_model_metric = Metric(df2_data["validate_y"], df2_all_features_pred_y)
print(f"Precision: {df2_all_features_model_metric.precision()}")
print(f"Recall: {df2_all_features_model_metric.recall()}")
print(f"Accuracy: {df2_all_features_model_metric.accuracy()}")
print(f"F1 Score: {df2_all_features_model_metric.f1_score()}")
display(df2_all_features_model_metric.confusion_matrix())

Precision: 0.8548387096774194
Recall: 0.7479838709677419
Accuracy: 0.8707903780068729
F1 Score: 0.7978494623655914


,Predicted Positive,Predicted Negative
Positive,371,125
Negative,63,896


### DF 3: Impute with mean only

In [ ]:
df3_data = get_experiment_data(data_imputed_mean_only)

#### Minimal

In [ ]:
df3_minimal_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df3_minimal_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df3_minimal_history = df3_minimal_model.fit(df3_data["train_x"]["minimal"], df3_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 2s 7ms/step - loss: 3.0394
Epoch 2/300
137/137 [==============================] - 1s 8ms/step - loss: 0.7026
Epoch 3/300
137/137 [==============================] - 1s 11ms/step - loss: 0.6855
Epoch 4/300
137/137 [==============================] - 1s 10ms/step - loss: 0.6711
Epoch 5/300
137/137 [==============================] - 1s 11ms/step - loss: 0.6494
Epoch 6/300
137/137 [==============================] - 1s 11ms/step - loss: 0.6481
Epoch 7/300
137/137 [==============================] - 1s 9ms/step - loss: 0.6484
Epoch 8/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6534
Epoch 9/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6533
Epoch 10/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6400
Epoch 11/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6456
Epoch 12/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6577
Epoch 13/

In [ ]:
df3_minimal_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_80 (Dense)            (None, 512)               4096      
                                                                 
 dense_81 (Dense)            (None, 512)               262656    
                                                                 
 dense_82 (Dense)            (None, 256)               131328    
                                                                 
 dense_83 (Dense)            (None, 256)               65792     
                                                                 
 dense_84 (Dense)            (None, 128)               32896     
                                                                 
 dense_85 (Dense)            (None, 64)                8256      
                                                                 
 dense_86 (Dense)            (None, 32)              

In [ ]:
df3_minimal_pred_y = df3_minimal_model.predict(df3_data["validate_x"]["minimal"])
df3_minimal_pred_y = np.where(df3_minimal_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 4ms/step


In [ ]:
df3_minimal_model_metric = Metric(df3_data["validate_y"], df3_minimal_pred_y)
print(f"Precision: {df3_minimal_model_metric.precision()}")
print(f"Recall: {df3_minimal_model_metric.recall()}")
print(f"Accuracy: {df3_minimal_model_metric.accuracy()}")
print(f"F1 Score: {df3_minimal_model_metric.f1_score()}")
display(df3_minimal_model_metric.confusion_matrix())

Precision: nan
Recall: 0.0
Accuracy: 0.6652920962199312
F1 Score: 0.0


,Predicted Positive,Predicted Negative
Positive,0,487
Negative,0,968


#### Baseline

In [ ]:
df3_baseline_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df3_baseline_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df3_baseline_history = df3_baseline_model.fit(df3_data["train_x"]["baseline"], df3_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 3s 10ms/step - loss: 2.3568
Epoch 2/300
137/137 [==============================] - 1s 11ms/step - loss: 0.6394
Epoch 3/300
137/137 [==============================] - 1s 10ms/step - loss: 0.6232
Epoch 4/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5601
Epoch 5/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5479
Epoch 6/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5309
Epoch 7/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5344
Epoch 8/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5141
Epoch 9/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5101
Epoch 10/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5145
Epoch 11/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5146
Epoch 12/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4995
Epoch 13/3

In [ ]:
df3_baseline_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_88 (Dense)            (None, 512)               17408     
                                                                 
 dense_89 (Dense)            (None, 512)               262656    
                                                                 
 dense_90 (Dense)            (None, 256)               131328    
                                                                 
 dense_91 (Dense)            (None, 256)               65792     
                                                                 
 dense_92 (Dense)            (None, 128)               32896     
                                                                 
 dense_93 (Dense)            (None, 64)                8256      
                                                                 
 dense_94 (Dense)            (None, 32)              

In [ ]:
df3_baseline_pred_y = df3_baseline_model.predict(df3_data["validate_x"]["baseline"])
df3_baseline_pred_y = np.where(df3_baseline_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 2ms/step


In [ ]:
df3_baseline_model_metric = Metric(df3_data["validate_y"], df3_baseline_pred_y)
print(f"Precision: {df3_baseline_model_metric.precision()}")
print(f"Recall: {df3_baseline_model_metric.recall()}")
print(f"Accuracy: {df3_baseline_model_metric.accuracy()}")
print(f"F1 Score: {df3_baseline_model_metric.f1_score()}")
display(df3_baseline_model_metric.confusion_matrix())

Precision: 0.6600790513833992
Recall: 0.6858316221765913
Accuracy: 0.7766323024054983
F1 Score: 0.6727089627391742


,Predicted Positive,Predicted Negative
Positive,334,153
Negative,172,796


#### With Weather History

In [ ]:
df3_weather_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df3_weather_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df3_weather_history = df3_weather_model.fit(df3_data["train_x"]["weather_hist"], df3_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 2s 7ms/step - loss: 1.4708
Epoch 2/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6132
Epoch 3/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5498
Epoch 4/300
137/137 [==============================] - 1s 9ms/step - loss: 0.5078
Epoch 5/300
137/137 [==============================] - 2s 11ms/step - loss: 0.4931
Epoch 6/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4877
Epoch 7/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4914
Epoch 8/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4730
Epoch 9/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4730
Epoch 10/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4697
Epoch 11/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4687
Epoch 12/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4505
Epoch 13/

In [ ]:
df3_weather_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_96 (Dense)            (None, 512)               37376     
                                                                 
 dense_97 (Dense)            (None, 512)               262656    
                                                                 
 dense_98 (Dense)            (None, 256)               131328    
                                                                 
 dense_99 (Dense)            (None, 256)               65792     
                                                                 
 dense_100 (Dense)           (None, 128)               32896     
                                                                 
 dense_101 (Dense)           (None, 64)                8256      
                                                                 
 dense_102 (Dense)           (None, 32)              

In [ ]:
df3_weather_pred_y = df3_weather_model.predict(df3_data["validate_x"]["weather_hist"])
df3_weather_pred_y = np.where(df3_weather_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 2ms/step


In [ ]:
df3_weather_model_metric = Metric(df3_data["validate_y"], df3_weather_pred_y)
print(f"Precision: {df3_weather_model_metric.precision()}")
print(f"Recall: {df3_weather_model_metric.recall()}")
print(f"Accuracy: {df3_weather_model_metric.accuracy()}")
print(f"F1 Score: {df3_weather_model_metric.f1_score()}")
display(df3_weather_model_metric.confusion_matrix())

Precision: 0.675564681724846
Recall: 0.675564681724846
Accuracy: 0.7828178694158076
F1 Score: 0.675564681724846


,Predicted Positive,Predicted Negative
Positive,329,158
Negative,158,810


#### With Fire History

In [ ]:
df3_fire_hist_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df3_fire_hist_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df3_fire_hist_history = df3_fire_hist_model.fit(df3_data["train_x"]["fire_hist"], df3_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 2s 7ms/step - loss: 3.9590
Epoch 2/300
137/137 [==============================] - 1s 7ms/step - loss: 1.0688
Epoch 3/300
137/137 [==============================] - 1s 7ms/step - loss: 1.0286
Epoch 4/300
137/137 [==============================] - 1s 7ms/step - loss: 0.5518
Epoch 5/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4947
Epoch 6/300
137/137 [==============================] - 1s 9ms/step - loss: 0.4603
Epoch 7/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4796
Epoch 8/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4111
Epoch 9/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4234
Epoch 10/300
137/137 [==============================] - 1s 11ms/step - loss: 0.3903
Epoch 11/300
137/137 [==============================] - 1s 8ms/step - loss: 0.3907
Epoch 12/300
137/137 [==============================] - 1s 7ms/step - loss: 0.4105
Epoch 13/

In [ ]:
df3_fire_hist_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_104 (Dense)           (None, 512)               18432     
                                                                 
 dense_105 (Dense)           (None, 512)               262656    
                                                                 
 dense_106 (Dense)           (None, 256)               131328    
                                                                 
 dense_107 (Dense)           (None, 256)               65792     
                                                                 
 dense_108 (Dense)           (None, 128)               32896     
                                                                 
 dense_109 (Dense)           (None, 64)                8256      
                                                                 
 dense_110 (Dense)           (None, 32)              

In [ ]:
df3_fire_hist_pred_y = df3_fire_hist_model.predict(df3_data["validate_x"]["fire_hist"])
df3_fire_hist_pred_y = np.where(df3_fire_hist_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 2ms/step


In [ ]:
df3_fire_hist_model_metric = Metric(df3_data["validate_y"], df3_fire_hist_pred_y)
print(f"Precision: {df3_fire_hist_model_metric.precision()}")
print(f"Recall: {df3_fire_hist_model_metric.recall()}")
print(f"Accuracy: {df3_fire_hist_model_metric.accuracy()}")
print(f"F1 Score: {df3_fire_hist_model_metric.f1_score()}")
display(df3_fire_hist_model_metric.confusion_matrix())

Precision: 0.7987679671457906
Recall: 0.7987679671457906
Accuracy: 0.8652920962199313
F1 Score: 0.7987679671457906


,Predicted Positive,Predicted Negative
Positive,389,98
Negative,98,870


#### With All Features

In [ ]:
df3_all_features_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, use_bias=True),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, use_bias=True)
])

In [ ]:
df3_all_features_model.compile(loss=tf.keras.metrics.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
df3_all_features_history = df3_all_features_model.fit(df3_data["train_x"]["all"], df3_data["train_y"], epochs=300)

Epoch 1/300
137/137 [==============================] - 2s 7ms/step - loss: 6.2706
Epoch 2/300
137/137 [==============================] - 1s 7ms/step - loss: 2.4636
Epoch 3/300
137/137 [==============================] - 1s 7ms/step - loss: 0.7639
Epoch 4/300
137/137 [==============================] - 1s 7ms/step - loss: 0.6806
Epoch 5/300
137/137 [==============================] - 1s 8ms/step - loss: 0.6053
Epoch 6/300
137/137 [==============================] - 1s 10ms/step - loss: 0.5306
Epoch 7/300
137/137 [==============================] - 1s 11ms/step - loss: 0.5091
Epoch 8/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4667
Epoch 9/300
137/137 [==============================] - 1s 11ms/step - loss: 0.4076
Epoch 10/300
137/137 [==============================] - 1s 9ms/step - loss: 0.5134
Epoch 11/300
137/137 [==============================] - 1s 7ms/step - loss: 0.3949
Epoch 12/300
137/137 [==============================] - 1s 7ms/step - loss: 0.3901
Epoch 13/

In [ ]:
df3_all_features_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_112 (Dense)           (None, 512)               38400     
                                                                 
 dense_113 (Dense)           (None, 512)               262656    
                                                                 
 dense_114 (Dense)           (None, 256)               131328    
                                                                 
 dense_115 (Dense)           (None, 256)               65792     
                                                                 
 dense_116 (Dense)           (None, 128)               32896     
                                                                 
 dense_117 (Dense)           (None, 64)                8256      
                                                                 
 dense_118 (Dense)           (None, 32)              

In [ ]:
df3_all_features_pred_y = df3_all_features_model.predict(df3_data["validate_x"]["all"])
df3_all_features_pred_y = np.where(df3_all_features_pred_y > .5, 1, 0)

46/46 [==============================] - 0s 2ms/step


In [ ]:
df3_all_features_model_metric = Metric(df3_data["validate_y"], df3_all_features_pred_y)
print(f"Precision: {df3_all_features_model_metric.precision()}")
print(f"Recall: {df3_all_features_model_metric.recall()}")
print(f"Accuracy: {df3_all_features_model_metric.accuracy()}")
print(f"F1 Score: {df3_all_features_model_metric.f1_score()}")
display(df3_all_features_model_metric.confusion_matrix())

Precision: 0.815845824411135
Recall: 0.7823408624229979
Accuracy: 0.8680412371134021
F1 Score: 0.7987421383647799


,Predicted Positive,Predicted Negative
Positive,381,106
Negative,86,882


## Compare Results

In [ ]:
compare_precision = pd.DataFrame([
        [df1_minimal_model_metric.precision(),
         df1_baseline_model_metric.precision(),
         df1_weather_model_metric.precision(),
         df1_fire_hist_model_metric.precision(),
         df1_all_features_model_metric.precision()],
        [df2_minimal_model_metric.precision(),
         df2_baseline_model_metric.precision(),
         df2_weather_model_metric.precision(),
         df2_fire_hist_model_metric.precision(),
         df2_all_features_model_metric.precision()],
        [df3_minimal_model_metric.precision(),
         df3_baseline_model_metric.precision(),
         df3_weather_model_metric.precision(),
         df3_fire_hist_model_metric.precision(),
         df3_all_features_model_metric.precision()]],
    columns=["minimal", "baseline", "weather_hist", "fire_hist", "all"],
    index=["df1", "df2", "df3"])
compare_precision

,minimal,baseline,weather_hist,fire_hist,all
df1,NaN,0.585831,0.568528,0.744737,0.788512
df2,NaN,0.655773,0.675676,0.801961,0.854839
df3,NaN,0.660079,0.675565,0.798768,0.815846


In [ ]:
compare_recall = pd.DataFrame([
        [df1_minimal_model_metric.recall(),
         df1_baseline_model_metric.recall(),
         df1_weather_model_metric.recall(),
         df1_fire_hist_model_metric.recall(),
         df1_all_features_model_metric.recall()],
        [df2_minimal_model_metric.recall(),
         df2_baseline_model_metric.recall(),
         df2_weather_model_metric.recall(),
         df2_fire_hist_model_metric.recall(),
         df2_all_features_model_metric.recall()],
        [df3_minimal_model_metric.recall(),
         df3_baseline_model_metric.recall(),
         df3_weather_model_metric.recall(),
         df3_fire_hist_model_metric.recall(),
         df3_all_features_model_metric.recall()]],
    columns=["minimal", "baseline", "weather_hist", "fire_hist", "all"],
    index=["df1", "df2", "df3"])
compare_recall

,minimal,baseline,weather_hist,fire_hist,all
df1,0.0,0.581081,0.605405,0.764865,0.816216
df2,0.0,0.606855,0.604839,0.824597,0.747984
df3,0.0,0.685832,0.675565,0.798768,0.782341


In [ ]:
compare_accuracy = pd.DataFrame([
        [df1_minimal_model_metric.accuracy(),
         df1_baseline_model_metric.accuracy(),
         df1_weather_model_metric.accuracy(),
         df1_fire_hist_model_metric.accuracy(),
         df1_all_features_model_metric.accuracy()],
        [df2_minimal_model_metric.accuracy(),
         df2_baseline_model_metric.accuracy(),
         df2_weather_model_metric.accuracy(),
         df2_fire_hist_model_metric.accuracy(),
         df2_all_features_model_metric.accuracy()],
        [df3_minimal_model_metric.accuracy(),
         df3_baseline_model_metric.accuracy(),
         df3_weather_model_metric.accuracy(),
         df3_fire_hist_model_metric.accuracy(),
         df3_all_features_model_metric.accuracy()]],
    columns=["minimal", "baseline", "weather_hist", "fire_hist", "all"],
    index=["df1", "df2", "df3"])
compare_accuracy

,minimal,baseline,weather_hist,fire_hist,all
df1,0.697218,0.748773,0.741408,0.849427,0.878069
df2,0.659107,0.757388,0.766323,0.870790,0.870790
df3,0.665292,0.776632,0.782818,0.865292,0.868041


In [ ]:
compare_f1_score = pd.DataFrame([
        [df1_minimal_model_metric.f1_score(),
         df1_baseline_model_metric.f1_score(),
         df1_weather_model_metric.f1_score(),
         df1_fire_hist_model_metric.f1_score(),
         df1_all_features_model_metric.f1_score()],
        [df2_minimal_model_metric.f1_score(),
         df2_baseline_model_metric.f1_score(),
         df2_weather_model_metric.f1_score(),
         df2_fire_hist_model_metric.f1_score(),
         df2_all_features_model_metric.f1_score()],
        [df3_minimal_model_metric.f1_score(),
         df3_baseline_model_metric.f1_score(),
         df3_weather_model_metric.f1_score(),
         df3_fire_hist_model_metric.f1_score(),
         df3_all_features_model_metric.f1_score()]],
    columns=["minimal", "baseline", "weather_hist", "fire_hist", "all"],
    index=["df1", "df2", "df3"])
compare_f1_score

,minimal,baseline,weather_hist,fire_hist,all
df1,0.0,0.583446,0.586387,0.754667,0.802125
df2,0.0,0.630366,0.638298,0.813121,0.797849
df3,0.0,0.672709,0.675565,0.798768,0.798742


### Save datasets to use again

In [ ]:
from datetime import datetime

In [ ]:
OUTFILE_NAME_TEMPLATE = "/content/drive/My Drive/ML6140  - Project/Model/train_test_split_data/{date}_{identifier}.csv"

In [ ]:
date = f"{datetime.now().date()}"

In [ ]:
def data_dict_to_csv(data_dict, name):
  pd.DataFrame(data_dict["train_x"]["all"]).to_csv(OUTFILE_NAME_TEMPLATE.format(
      date=date,
      identifier=f"{name}_train_x"),
    header=data_dropped_nans.drop(["class_label"], axis=1).columns, index=False)
  pd.DataFrame(data_dict["train_y"]).to_csv(OUTFILE_NAME_TEMPLATE.format(
      date=date,
      identifier=f"{name}_train_y"),
    header=["class_label"], index=False)
  pd.DataFrame(data_dict["validate_x"]["all"]).to_csv(OUTFILE_NAME_TEMPLATE.format(
      date=date,
      identifier=f"{name}_validate_x"),
    header=data_dropped_nans.drop(["class_label"], axis=1).columns, index=False)
  pd.DataFrame(data_dict["validate_y"]).to_csv(OUTFILE_NAME_TEMPLATE.format(
      date=date,
      identifier=f"{name}_validate_y"),
    header=["class_label"], index=False)

In [ ]:
data_dict_to_csv(df1_data, "df1")
data_dict_to_csv(df2_data, "df2")
data_dict_to_csv(df3_data, "df3")